In [2]:
import cv2
import numpy as np
import sys
from pathlib import Path
from segment_anything import sam_model_registry, SamPredictor

In [3]:
sam_checkpoint = "weights/sam_vit_h_4b8939.pth"
model_type = "vit_h"
# device = "cuda"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
# sam.to(device=device)
predictor = SamPredictor(sam)

In [4]:
image_file = Path(r'..\data\rgb_images\11-1\DSC_0537 (Pot 2) .JPG')
image = cv2.imread(str(image_file))

In [5]:
input_points = np.array([[200,200]])
input_labels = np.array([1])

In [6]:
predictor.set_image(image)

In [9]:
masks, scores, logits = predictor.predict(
    point_coords=np.array(input_points),
    point_labels=np.array(input_labels),
    multimask_output=True,
)

In [8]:
for mask in masks:
    color = np.array([30/255, 144/255, 255/255], dtype=np.float32)
    h, w = mask.shape[-2:]
    mask = mask.reshape(h,w,1) * color.reshape(1,1,-1)
    image = cv2.addWeighted(image.astype(np.float32)/, 0.5, mask, 0.5, 0)

In [13]:
mask = np.any(masks, axis=0)
mask_3d = np.repeat(mask[:, :, np.newaxis], 3, axis=2)
mask_image = np.copy(image)
color = np.array([255, 144, 30], dtype=np.uint8)
mask_image[mask] = color
image_processed = cv2.addWeighted(image, 0.5, mask_image, 0.5, 0)

In [14]:
bb = np.array([[True, False], [True,False]])
print(bb)
print(bb.astype(np.uint8))

[[ True False]
 [ True False]]
[[1 0]
 [1 0]]
